# Machine Learning Tutorial

We will build a Deep Neural Network to classify sprinklings of various discrete spacetimes (causal sets). We will use Google's TensorFlow (https://www.tensorflow.org) to build the DNN, and then use data that I will provide to train and test the DNN.

## TensorFlow

The first step is to install tensorflow. You can follow the instructions in the link above. Should you have any problems with the installation we can try to fix it during our discussion (with Jef's help!). 

As well as tensorflow we will need the numpy and pandas libraries. The former should already be part of your python installation, while if you don't have the latter you can install it using pip: 

$pip install pandas

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

/Users/dbenincasa/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/dbenincasa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Import and Parse Data

We begin by importing and parsing the datasets.

In [ ]:
basedir = '/Users/dbenincasa/Documents/machine-learning/tutorials'